#### Notes

Running the script below to create events for 2022-2023 academic calendar ([link](https://docs.google.com/spreadsheets/d/1a0ov5ctf7XA8DpAWxyScy99tJVCp90GKvwv_mmWbIHQ/edit#gid=1480204723))

In [1]:
from __future__ import print_function
import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import numpy as np
import os.path
import pickle

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
print(datetime.datetime.today())

2022-08-13 16:35:43.570523


<IPython.core.display.Javascript object>

In [ ]:
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

In [10]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar',
          'https://www.googleapis.com/auth/spreadsheets.readonly'
]

calendarID="c_822m07aktpjne3gfbr7cj8gjks@group.calendar.google.com"

"""Shows basic usage of the Google Calendar API.
Prints the start and name of the next 10 events on the user's calendar.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.

if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('calendar', 'v3', credentials=creds)

<IPython.core.display.Javascript object>

In [11]:
# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId=calendarID, timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])


Getting the upcoming 10 events
No upcoming events found.


<IPython.core.display.Javascript object>

### Pull data from Google sheet

In [15]:
# obtained from google sheet url:
# https://docs.google.com/spreadsheets/d/1a0ov5ctf7XA8DpAWxyScy99tJVCp90GKvwv_mmWbIHQ/edit#gid=1480204723
SPREADSHEET_ID = "1a0ov5ctf7XA8DpAWxyScy99tJVCp90GKvwv_mmWbIHQ"

<IPython.core.display.Javascript object>

In [21]:
from googleapiclient.discovery import build
def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
data = pull_sheet_data(SCOPES, SPREADSHEET_ID , 'Computer_Readable!A1:G')

COMPLETE: Data copied


<IPython.core.display.Javascript object>

In [78]:
# sanity check 
print(len(data))
data[1]

185


['9/17/2022',
 '10 am - 11 am',
 'WCT',
 'wct-instructors, students2024',
 'Writing and Critical Thinking',
 '4']

<IPython.core.display.Javascript object>

In [24]:
import pandas as pd
df = pd.DataFrame(data[1:], columns=data[0])

print(df.shape)
df.head(2)

(184, 6)


,Date,Time,Calendar Assignment,Share with group,Event Title,Color
0,9/17/2022,10 am - 11 am,WCT,"wct-instructors, students2024",Writing and Critical Thinking,4
1,9/17/2022,11 am - 12 pm,Math Matters,"math-instructors, students2024",Math Matters,3


<IPython.core.display.Javascript object>

In [25]:
df['start']=df.Time.str.split(' - ').map(lambda x: x[0])
df['end']=df.Time.str.split(' - ').map(lambda x: x[1])

<IPython.core.display.Javascript object>

In [26]:
df.head(2)

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end
0,9/17/2022,10 am - 11 am,WCT,"wct-instructors, students2024",Writing and Critical Thinking,4,10 am,11 am
1,9/17/2022,11 am - 12 pm,Math Matters,"math-instructors, students2024",Math Matters,3,11 am,12 pm


<IPython.core.display.Javascript object>

In [39]:
eventservice=service.events()

<IPython.core.display.Javascript object>

In [69]:
df["calendarEventID"] = df["calendarEventID"].replace(" ", None)

<IPython.core.display.Javascript object>

In [70]:
df.head()

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end,calendarEventID
0,9/17/2022,10 am - 11 am,WCT,"wct-instructors, students2024",Writing and Critical Thinking,4,10 am,11 am,tpeajj87r1itb9klee0oc3i86s
1,9/17/2022,11 am - 12 pm,Math Matters,"math-instructors, students2024",Math Matters,3,11 am,12 pm,None
2,9/17/2022,10 am - 12 pm,Test prep,"testprep-instructors, students2023",Test Prep,5,10 am,12 pm,None
3,9/17/2022,10 am - 12 pm,Senior Enrichment,"senior-enrichment-instructors, students2022",Senior Enrichment,6,10 am,12 pm,None
4,9/17/2022,12 pm - 2 pm,Sophomore Mentoring,"2024mentors, students2024",Sophomore Mentoring,7,12 pm,2 pm,None


<IPython.core.display.Javascript object>

In [72]:
import time
for k,row in df.iterrows():
    try:
        starttime=datetime.datetime.strptime(row.Date + " " + row.start, '%m/%d/%Y %I:%M %p')
    except ValueError:
        starttime=datetime.datetime.strptime(row.Date + " " + row.start, '%m/%d/%Y %I %p')
        
    try:
        endtime=datetime.datetime.strptime(row.Date + " " + row.end, '%m/%d/%Y %I:%M %p')
    except ValueError:
        endtime=datetime.datetime.strptime(row.Date + " " + row.end, '%m/%d/%Y %I %p')
    
    body={
        "description": row['Event Title'],
        "summary":  row['Event Title'],
        "start":{
            "dateTime": starttime.isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "end":{
            "dateTime": endtime.isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "colorId": f"{row.Color}",
        "guestsCanInviteOthers": False

    }
    
    
    if row.calendarEventID:
        pass
    else:
        r=eventservice.insert(calendarId=calendarID,
                             body=body)
        response=r.execute()
        df.loc[k, 'calendarEventID']=response["id"]
        time.sleep(.2)

<IPython.core.display.Javascript object>

In [73]:
df.head()

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end,calendarEventID
0,9/17/2022,10 am - 11 am,WCT,"wct-instructors, students2024",Writing and Critical Thinking,4,10 am,11 am,tpeajj87r1itb9klee0oc3i86s
1,9/17/2022,11 am - 12 pm,Math Matters,"math-instructors, students2024",Math Matters,3,11 am,12 pm,kqnqjt1a02l9sgslohrjmc6848
2,9/17/2022,10 am - 12 pm,Test prep,"testprep-instructors, students2023",Test Prep,5,10 am,12 pm,2iksc0qp0greso2i49cme2l7u8
3,9/17/2022,10 am - 12 pm,Senior Enrichment,"senior-enrichment-instructors, students2022",Senior Enrichment,6,10 am,12 pm,b4q4kc3e1p2pfu3q5pvv7lduuk
4,9/17/2022,12 pm - 2 pm,Sophomore Mentoring,"2024mentors, students2024",Sophomore Mentoring,7,12 pm,2 pm,6si8g2rov0fv5rapc56tsukhvo


<IPython.core.display.Javascript object>

In [75]:
for k,row in df.iterrows():
    emails=map(lambda x: x.replace(' ','') ,row['Share with group'].split(','))
    event = eventservice.get(calendarId=calendarID, eventId=row.calendarEventID).execute()
    attendees=event.get('attendees',[])
    for email in emails:
        try:
            invite=next(a for a in attendees if a['email']==email + "@mindsmatterseattle.org")
        except StopIteration:
            print(f'inviting {email} to {event["summary"]}')
            invite = {
                'email': f'{email}@mindsmatterseattle.org'
            }
            attendees.append(invite)

    event['attendees'] =  attendees 
    eventservice.update(calendarId=calendarID, eventId=row.calendarEventID, body=event).execute()
   

inviting wct-instructors to Writing and Critical Thinking
inviting students2024 to Writing and Critical Thinking
inviting math-instructors to Math Matters
inviting students2024 to Math Matters
inviting testprep-instructors to Test Prep
inviting students2023 to Test Prep
inviting senior-enrichment-instructors to Senior Enrichment
inviting students2022 to Senior Enrichment
inviting 2024mentors to Sophomore Mentoring
inviting students2024 to Sophomore Mentoring
inviting 2023mentors to Junior Mentoring
inviting students2023 to Junior Mentoring
inviting 2022mentors to Senior Mentoring
inviting students2022 to Senior Mentoring
inviting ec to EC Meeting
inviting wct-instructors to Writing and Critical Thinking
inviting students2024 to Writing and Critical Thinking
inviting math-instructors to Math Matters
inviting students2024 to Math Matters
inviting testprep-instructors to Test Prep
inviting students2023 to Test Prep
inviting senior-enrichment-instructors to Senior Enrichment
inviting stude

<IPython.core.display.Javascript object>

In [76]:
df.to_csv('calendar_events.csv')

<IPython.core.display.Javascript object>

In [77]:
df.head()

,Date,Time,Calendar Assignment,Share with group,Event Title,Color,start,end,calendarEventID
0,9/17/2022,10 am - 11 am,WCT,"wct-instructors, students2024",Writing and Critical Thinking,4,10 am,11 am,tpeajj87r1itb9klee0oc3i86s
1,9/17/2022,11 am - 12 pm,Math Matters,"math-instructors, students2024",Math Matters,3,11 am,12 pm,kqnqjt1a02l9sgslohrjmc6848
2,9/17/2022,10 am - 12 pm,Test prep,"testprep-instructors, students2023",Test Prep,5,10 am,12 pm,2iksc0qp0greso2i49cme2l7u8
3,9/17/2022,10 am - 12 pm,Senior Enrichment,"senior-enrichment-instructors, students2022",Senior Enrichment,6,10 am,12 pm,b4q4kc3e1p2pfu3q5pvv7lduuk
4,9/17/2022,12 pm - 2 pm,Sophomore Mentoring,"2024mentors, students2024",Sophomore Mentoring,7,12 pm,2 pm,6si8g2rov0fv5rapc56tsukhvo


<IPython.core.display.Javascript object>